In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [ ]:
pip install quandl

In [ ]:
pip install cvxportfolio

In [ ]:
pip install yahoo_fin

In [ ]:
pip install requests_html

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import quandl
import yahoo_fin.stock_info as si

import cvxportfolio as cp

/usr/local/lib/python3.6/dist-packages/cvxportfolio/policies.py:42: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  def get_trades(self, portfolio, t=pd.datetime.today()):
/usr/local/lib/python3.6/dist-packages/cvxportfolio/policies.py:60: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  def get_trades(self, portfolio, t=pd.datetime.today()):
/usr/local/lib/python3.6/dist-packages/cvxportfolio/policies.py:74: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  def get_trades(self, portfolio, t=pd.datetime.today()):
/usr/local/lib/python3.6/dist-packages/cvxportfolio/policies.py:107: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
 

In [ ]:
import importlib
importlib.reload(cp)

<module 'cvxportfolio' from '/usr/local/lib/python3.6/dist-packages/cvxportfolio/__init__.py'>

Download the problem data from Quandl. We select four liquid stocks, and the risk-free rate.

# Universe
We define the universe of tradeable assets for HFAC. HFAC only trades micro-cap and small-cap equities (<$5 billion market cap).

In [ ]:
ticker_data = {}
for ticker in si.tickers_dow():
  ticker_data[ticker] = si.get_quote_table(ticker)
ticker_data

In [ ]:
for ticker, ticker_info in ticker_data.items():
  print(f'{ticker} Market Cap: {ticker_info["Market Cap"]}')

AAPL Market Cap: 2.036T
AMGN Market Cap: 138.06B
AXP Market Cap: 84.469B
BA Market Cap: 94.461B
CAT Market Cap: 91.379B
CRM Market Cap: 235.28B
CSCO Market Cap: 170.015B
CVX Market Cap: 140.314B
DIS Market Cap: 229.153B
DOW Market Cap: 36.537B
GS Market Cap: 70.952B
HD Market Cap: 309.654B
HON Market Cap: 122.714B
IBM Market Cap: 112.151B
INTC Market Cap: 230.342B
JNJ Market Cap: 389.921B
JPM Market Cap: 309.423B
KO Market Cap: 214.901B
MCD Market Cap: 170.675B
MMM Market Cap: 98.482B
MRK Market Cap: 201.909B
MSFT Market Cap: 1.662T
NKE Market Cap: 200.938B
PG Market Cap: 359.476B
TRV Market Cap: 28.692B
UNH Market Cap: 313.516B
V Market Cap: 439.659B
VZ Market Cap: 240.214B
WBA Market Cap: 32.394B
WMT Market Cap: 410.072B


In [ ]:
tickers = ['AMZN', 'GOOGL', 'TSLA', 'NKE']
start_date='2012-01-01'
end_date='2020-09-31'
returns = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'].pct_change())
                for ticker in tickers]))
returns[["USDOLLAR"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

LimitExceededError: ignored

In [ ]:
amzn_data = quandl.get('WIKI/AMZN', start_date=start_date, end_date=end_date)

In [ ]:
amzn_data = quandl.get_table('WIKI/AMZN')

LimitExceededError: ignored

We compute rolling estimates of the first and second moments of the returns using a window of 250 days. We shift them by one unit (so at every day we present the optimizer with only past data).

In [ ]:
r_hat = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
Sigma_hat = returns.rolling(window=250, min_periods=250, closed='neither').cov().dropna()

r_hat.tail()

# Transaction Costs and Holding Costs

Here we define the transaction cost and holding cost model (sections 2.3 and 2.4 [of the paper](https://web.stanford.edu/~boyd/papers/cvx_portfolio.html)). The data can be expressed 
as 
- a scalar (like we're doing here), the same value for all assets and all time periods;
- a Pandas Series indexed by the asset names, for asset-specific values; 
- a Pandas DataFrame indexed by timestamps with asset names as columns, for values that vary by asset and in time.

In [ ]:
tcost_model=cp.TcostModel(half_spread=10E-4) # need: sigma, volumes (source from api)
hcost_model=cp.HcostModel(borrow_costs=1E-4) # need: dividends (source from api)

# Trading and Holding Constraints

In [ ]:
long_only = cp.LongOnly()

# Risk Model

In [ ]:
risk_model = cp.FullSigma(Sigma_hat.reset_index(level=1, drop=True))

# Full Optimization Policy

We define the single period optimization policy (section 4 [of the paper](https://web.stanford.edu/~boyd/papers/cvx_portfolio.html)). 

In [ ]:
gamma_risk, gamma_trade, gamma_hold = 5., 1., 1.

spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[gamma_risk*risk_model, gamma_trade*tcost_model, gamma_hold*hcost_model],
                                constraints=[leverage_limit])

We run a backtest, which returns a result object. By calling its summary method we get some basic statistics.

In [ ]:
market_sim=cp.MarketSimulator(returns, [tcost_model, hcost_model], cash_key='USDOLLAR') 
init_portfolio = pd.Series(index=returns.columns, data=250000.)
init_portfolio.USDOLLAR = 0
results = market_sim.run_multiple_backtest(init_portfolio,
                               start_time='2013-01-03',  end_time='2016-12-31',  
                               policies=[spo_policy, cp.Hold()], parallel=False)
results[0].summary()

The total value of the portfolio in time.

In [ ]:
results[0].v.plot(figsize=(12,5))
results[1].v.plot(figsize=(12,5))

The weights vector of the portfolio in time.

In [ ]:
results[0].w.plot(figsize=(12,6))